In [1]:
import os
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.models import Model
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [2]:
import sys
print(sys.version)

3.6.2 |Anaconda custom (64-bit)| (default, Oct  5 2017, 07:59:26) 
[GCC 7.2.0]


In [3]:
import tensorflow as tf

In [4]:
import pandas as pd

In [5]:
tf.__version__

'1.12.0'

In [6]:
model = VGG16(weights='imagenet',include_top = True)

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [8]:
import numpy as np
import matplotlib.pyplot as plt

def load_image(path):
    img = image.load_img(path, target_size = model.input_shape[1:3])
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

#img, x = load_image("/58/204763545.jpg")
#print("shape of x: ", x.shape)
#print("data type: ", x.dtype)
#plt.imshow(img)

In [9]:
feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)
feat_extractor.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [9]:
feat = feat_extractor.predict(x)
plt.figure(figsize=(16,4))
plt.plot(feat[0])

NameError: name 'x' is not defined

In [10]:
from PIL import Image
#NOTE: CHANGE THIS IF YOU WANT TO SWITCH DIR
images_path = 'CrowdFlower/EntrywayImages'
image_extensions = ['.jpg', '.png', '.jpeg']
images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(images_path)
         for f in filenames if os.path.splitext(f)[1].lower() in image_extensions]

betterImages = []

for i, item in enumerate(images):
    if i % 1000 == 0:
        print(i)
    try:
        im = Image.open(item)
        betterImages.append(item)
    except IOError:
        continue
        
len(betterImages)

0
1000


1776

In [11]:
import time
tic = time.clock()

features = []
for i, image_path in enumerate(betterImages):
    if i % 500 == 0:
        toc = time.clock()
        elap = toc-tic;
        print("analyzing image %d / %d. Time: %4.4f seconds." % (i, len(betterImages), elap))
        tic = time.clock()
    img, x = load_image(image_path);
    feat = feat_extractor.predict(x)[0]
    features.append(feat)
    
print('finished extracting features for %d images' % len(images))

analyzing image 0 / 1776. Time: 0.0001 seconds.
analyzing image 500 / 1776. Time: 11.7354 seconds.
analyzing image 1000 / 1776. Time: 9.5556 seconds.
analyzing image 1500 / 1776. Time: 9.5902 seconds.
finished extracting features for 1776 images


In [12]:
len(features)

1776

In [13]:
from sklearn.decomposition import PCA

features = np.array(features)
pca = PCA(n_components=300)
pca.fit(features)

PCA(copy=True, iterated_power='auto', n_components=300, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [14]:
pca_features = pca.transform(features)
pca_features.shape

(1776, 300)

In [17]:
import pickle

with open('similarFeatures2.pkl', 'wb') as f:
    pickle.dump(pca_features, f)

#with open('similarFeatures.pkl', 'rb') as f:
    #pca_features = pickle.load(f)

In [15]:
from scipy.spatial import distance

In [16]:
########filtering images from the room classification set
#and then copying into a new directory


count = 0
pairs = list(zip(pca_features,betterImages))
filteredImages = betterImages

print(len(filteredImages))

for i,featureVec in enumerate(pca_features):
    print(i, len(pairs[i+1:]))
    for j, x in enumerate(pairs[i+1:]):
        if distance.cosine(featureVec, x[0]) < 0.02:
            if x[1] in filteredImages:
                filteredImages.remove(x[1])
    print(len(filteredImages))

1776
0 1775
1776
1 1774
1775
2 1773
1775
3 1772
1775
4 1771
1774
5 1770
1773
6 1769
1773
7 1768
1772
8 1767
1771
9 1766
1770
10 1765
1770
11 1764
1770
12 1763
1769
13 1762
1769
14 1761
1769
15 1760
1769
16 1759
1768
17 1758
1767
18 1757
1767
19 1756
1766
20 1755
1766
21 1754
1766
22 1753
1766
23 1752
1766
24 1751
1763
25 1750
1763
26 1749
1762
27 1748
1761
28 1747
1761
29 1746
1760
30 1745
1759
31 1744
1759
32 1743
1758
33 1742
1757
34 1741
1757
35 1740
1757
36 1739
1756
37 1738
1756
38 1737
1755
39 1736
1754
40 1735
1753
41 1734
1752
42 1733
1752
43 1732
1751
44 1731
1750
45 1730
1749
46 1729
1749
47 1728
1749
48 1727
1748
49 1726
1748
50 1725
1748
51 1724
1748
52 1723
1747
53 1722
1746
54 1721
1746
55 1720
1746
56 1719
1743
57 1718
1742
58 1717
1741
59 1716
1740
60 1715
1739
61 1714
1739
62 1713
1739
63 1712
1738
64 1711
1738
65 1710
1737
66 1709
1736
67 1708
1735
68 1707
1735
69 1706
1735
70 1705
1735
71 1704
1734
72 1703
1734
73 1702
1734
74 1701
1734
75 1700
1734
76 1699
1733
77 1

1513
594 1181
1512
595 1180
1512
596 1179
1512
597 1178
1512
598 1177
1512
599 1176
1512
600 1175
1512
601 1174
1511
602 1173
1511
603 1172
1511
604 1171
1510
605 1170
1510
606 1169
1510
607 1168
1510
608 1167
1510
609 1166
1510
610 1165
1510
611 1164
1510
612 1163
1510
613 1162
1510
614 1161
1509
615 1160
1508
616 1159
1508
617 1158
1508
618 1157
1508
619 1156
1508
620 1155
1508
621 1154
1508
622 1153
1508
623 1152
1508
624 1151
1508
625 1150
1507
626 1149
1506
627 1148
1505
628 1147
1505
629 1146
1505
630 1145
1504
631 1144
1504
632 1143
1503
633 1142
1503
634 1141
1503
635 1140
1503
636 1139
1503
637 1138
1503
638 1137
1503
639 1136
1503
640 1135
1503
641 1134
1503
642 1133
1503
643 1132
1503
644 1131
1503
645 1130
1502
646 1129
1500
647 1128
1500
648 1127
1500
649 1126
1500
650 1125
1500
651 1124
1499
652 1123
1499
653 1122
1499
654 1121
1498
655 1120
1498
656 1119
1497
657 1118
1497
658 1117
1497
659 1116
1497
660 1115
1497
661 1114
1497
662 1113
1497
663 1112
1497
664 1111
1497
6

1377
1201 574
1377
1202 573
1377
1203 572
1377
1204 571
1377
1205 570
1377
1206 569
1377
1207 568
1377
1208 567
1377
1209 566
1377
1210 565
1377
1211 564
1377
1212 563
1376
1213 562
1376
1214 561
1375
1215 560
1375
1216 559
1375
1217 558
1375
1218 557
1375
1219 556
1375
1220 555
1375
1221 554
1374
1222 553
1374
1223 552
1374
1224 551
1373
1225 550
1373
1226 549
1373
1227 548
1373
1228 547
1373
1229 546
1373
1230 545
1372
1231 544
1372
1232 543
1372
1233 542
1372
1234 541
1372
1235 540
1372
1236 539
1371
1237 538
1371
1238 537
1371
1239 536
1371
1240 535
1371
1241 534
1371
1242 533
1371
1243 532
1371
1244 531
1370
1245 530
1370
1246 529
1370
1247 528
1370
1248 527
1370
1249 526
1370
1250 525
1369
1251 524
1369
1252 523
1369
1253 522
1369
1254 521
1369
1255 520
1369
1256 519
1369
1257 518
1369
1258 517
1369
1259 516
1368
1260 515
1367
1261 514
1367
1262 513
1367
1263 512
1367
1264 511
1367
1265 510
1367
1266 509
1367
1267 508
1367
1268 507
1367
1269 506
1367
1270 505
1367
1271 504
1367
1

In [53]:
with open('filteredBathroomImages.pkl', 'wb') as f:
    pickle.dump(filteredImages, f)
    
#with open('RoomClassification/filteredRoomImages.pkl', 'rb') as f:
    #pca_features = pickle.load(f)

NameError: name 'pickle' is not defined

In [60]:
import shutil
for i,x in enumerate(filteredImages):
    if (i % 100 == 0):
        print(i)
    shutil.copy(x,'RoomClassification/filteredImages/' + x.split('/')[2] + '/' + x.split('/')[3])

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300


In [17]:
def get_imageID(path):
    return path.split('.')[0].split('/')[2]



In [32]:
####similar images on Bedroom#######
bedroomImages_path = 'Bedroom/BedroomSamples'
image_extensions = ['.jpg', '.png', '.jpeg']
bedroomImages = [os.path.join(dp, f) for dp, dn, filenames in os.walk(bedroomImages_path)
         for f in filenames if os.path.splitext(f)[1].lower() in image_extensions]
len(bedroomImages)

14261

In [235]:
import time
tic = time.clock()

bedroomFeatures = []
for i, image_path in enumerate(bedroomImages):
    if i % 500 == 0:
        toc = time.clock()
        elap = toc-tic;
        print("analyzing image %d / %d. Time: %4.4f seconds." % (i, len(bedroomImages), elap))
        tic = time.clock()
    img, x = load_image(image_path);
    feat = feat_extractor.predict(x)[0]
    bedroomFeatures.append(feat)
    
print('finished extracting features for %d images' % len(evalImages))

analyzing image 0 / 14261. Time: 0.0002 seconds.
analyzing image 500 / 14261. Time: 692.9845 seconds.
analyzing image 1000 / 14261. Time: 683.2109 seconds.
analyzing image 1500 / 14261. Time: 672.3662 seconds.
analyzing image 2000 / 14261. Time: 685.1840 seconds.
analyzing image 2500 / 14261. Time: 668.5611 seconds.
analyzing image 3000 / 14261. Time: 669.6629 seconds.
analyzing image 3500 / 14261. Time: 677.6144 seconds.
analyzing image 4000 / 14261. Time: 713.4128 seconds.
analyzing image 4500 / 14261. Time: 675.4799 seconds.
analyzing image 5000 / 14261. Time: 701.3383 seconds.
analyzing image 5500 / 14261. Time: 717.7468 seconds.
analyzing image 6000 / 14261. Time: 674.6459 seconds.
analyzing image 6500 / 14261. Time: 671.9952 seconds.
analyzing image 7000 / 14261. Time: 675.4598 seconds.
analyzing image 7500 / 14261. Time: 674.7124 seconds.
analyzing image 8000 / 14261. Time: 679.6006 seconds.
analyzing image 8500 / 14261. Time: 687.4811 seconds.
analyzing image 9000 / 14261. Time

In [236]:
from sklearn.decomposition import PCA

bedroomFeatures = np.array(bedroomFeatures)
pca = PCA(n_components=300)
pca.fit(bedroomFeatures)

PCA(copy=True, iterated_power='auto', n_components=300, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [237]:
pca_bedroomFeatures = pca.transform(bedroomFeatures)
pca_bedroomFeatures.shape

(14261, 300)

In [15]:
import pickle
from scipy.spatial import distance
#with open('bedroomFeatures.pkl', 'wb') as f:
    #pickle.dump(pca_bedroomFeatures, f)
    
with open('bedroomFeatures.pkl', 'rb') as f:
    pca_bedroomFeatures = pickle.load(f)

In [18]:
def get_imageID(path):
    return path.split('.')[0].split('/')[-1]

In [34]:
count = 0
bedroomPairs = list(zip(pca_bedroomFeatures,bedroomImages))
filteredBedroomImages = bedroomImages

print(len(filteredBedroomImages))

for i,featureVec in enumerate(pca_bedroomFeatures):
    print(i, len(bedroomPairs[i+1:]))
    for j, x in enumerate(bedroomPairs[i+1:]):
        if distance.cosine(featureVec, x[0]) < 0.03:
            if x[1] in filteredBedroomImages:
                filteredBedroomImages.remove(x[1])
    print(len(filteredBedroomImages))

14261
0 14260
14261
1 14259
14260
2 14258
14258
3 14257
14257
4 14256
14255
5 14255
14255
6 14254
14253
7 14253
14253
8 14252
14252
9 14251
14252
10 14250
14252
11 14249
14252
12 14248
14251
13 14247
14251
14 14246
14251
15 14245
14250
16 14244
14248
17 14243
14248
18 14242
14248
19 14241
14248
20 14240
14248
21 14239
14248
22 14238
14246
23 14237
14246
24 14236
14246
25 14235
14243
26 14234
14242
27 14233
14242
28 14232
14241
29 14231
14241
30 14230
14236
31 14229
14235
32 14228
14235
33 14227
14233
34 14226
14231
35 14225
14231
36 14224
14227
37 14223
14226
38 14222
14226
39 14221
13736
40 14220
13736
41 14219
13736
42 14218
13736
43 14217
13724
44 14216
13724
45 14215
13723
46 14214
13723
47 14213
13700
48 14212
13700
49 14211
13700
50 14210
13698
51 14209
13698
52 14208
13698
53 14207
13698
54 14206
13698
55 14205
13698
56 14204
13698
57 14203
13697
58 14202
13695
59 14201
13694
60 14200
13692
61 14199
13692
62 14198
13692
63 14197
13692
64 14196
13688
65 14195
13688
66 14194
13687

13257
519 13741
13251
520 13740
13251
521 13739
13251
522 13738
13251
523 13737
13251
524 13736
13251
525 13735
13251
526 13734
13249
527 13733
13249
528 13732
13249
529 13731
13249
530 13730
13249
531 13729
13249
532 13728
13248
533 13727
13248
534 13726
13248
535 13725
13247
536 13724
13246
537 13723
13246
538 13722
13246
539 13721
13243
540 13720
13243
541 13719
13243
542 13718
13242
543 13717
13242
544 13716
13242
545 13715
13242
546 13714
13242
547 13713
13242
548 13712
13242
549 13711
13242
550 13710
13242
551 13709
13241
552 13708
13241
553 13707
13241
554 13706
13241
555 13705
13241
556 13704
13241
557 13703
13239
558 13702
13239
559 13701
13239
560 13700
13239
561 13699
13239
562 13698
13239
563 13697
13237
564 13696
13237
565 13695
13236
566 13694
13236
567 13693
13236
568 13692
13236
569 13691
13228
570 13690
13227
571 13689
13227
572 13688
13227
573 13687
13227
574 13686
13227
575 13685
13227
576 13684
13227
577 13683
13227
578 13682
13227
579 13681
13227
580 13680
13227
58

12951
1030 13230
12951
1031 13229
12951
1032 13228
12951
1033 13227
12951
1034 13226
12951
1035 13225
12951
1036 13224
12951
1037 13223
12949
1038 13222
12949
1039 13221
12949
1040 13220
12949
1041 13219
12949
1042 13218
12949
1043 13217
12949
1044 13216
12949
1045 13215
12949
1046 13214
12949
1047 13213
12949
1048 13212
12947
1049 13211
12945
1050 13210
12945
1051 13209
12944
1052 13208
12944
1053 13207
12944
1054 13206
12942
1055 13205
12941
1056 13204
12941
1057 13203
12941
1058 13202
12939
1059 13201
12939
1060 13200
12939
1061 13199
12939
1062 13198
12939
1063 13197
12938
1064 13196
12937
1065 13195
12936
1066 13194
12934
1067 13193
12934
1068 13192
12932
1069 13191
12932
1070 13190
12931
1071 13189
12929
1072 13188
12929
1073 13187
12929
1074 13186
12929
1075 13185
12929
1076 13184
12929
1077 13183
12929
1078 13182
12929
1079 13181
12927
1080 13180
12926
1081 13179
12924
1082 13178
12924
1083 13177
12924
1084 13176
12924
1085 13175
12922
1086 13174
12917
1087 13173
12917
1088 131

12724
1512 12748
12723
1513 12747
12719
1514 12746
12718
1515 12745
12718
1516 12744
12718
1517 12743
12718
1518 12742
12718
1519 12741
12716
1520 12740
12716
1521 12739
12716
1522 12738
12716
1523 12737
12716
1524 12736
12716
1525 12735
12716
1526 12734
12716
1527 12733
12716
1528 12732
12713
1529 12731
12713
1530 12730
12713
1531 12729
12713
1532 12728
12713
1533 12727
12712
1534 12726
12712
1535 12725
12711
1536 12724
12711
1537 12723
12711
1538 12722
12710
1539 12721
12710
1540 12720
12710
1541 12719
12710
1542 12718
12709
1543 12717
12709
1544 12716
12709
1545 12715
12709
1546 12714
12709
1547 12713
12709
1548 12712
12709
1549 12711
12709
1550 12710
12709
1551 12709
12709
1552 12708
12709
1553 12707
12709
1554 12706
12708
1555 12705
12708
1556 12704
12708
1557 12703
12708
1558 12702
12707
1559 12701
12707
1560 12700
12707
1561 12699
12705
1562 12698
12705
1563 12697
12704
1564 12696
12704
1565 12695
12703
1566 12694
12703
1567 12693
12701
1568 12692
12701
1569 12691
12701
1570 126

12518
1994 12266
12518
1995 12265
12518
1996 12264
12518
1997 12263
12518
1998 12262
12518
1999 12261
12518
2000 12260
12518
2001 12259
12516
2002 12258
12516
2003 12257
12516
2004 12256
12516
2005 12255
12516
2006 12254
12516
2007 12253
12516
2008 12252
12516
2009 12251
12515
2010 12250
12515
2011 12249
12514
2012 12248
12513
2013 12247
12512
2014 12246
12512
2015 12245
12512
2016 12244
12512
2017 12243
12512
2018 12242
12512
2019 12241
12512
2020 12240
12512
2021 12239
12512
2022 12238
12512
2023 12237
12510
2024 12236
12510
2025 12235
12510
2026 12234
12510
2027 12233
12510
2028 12232
12510
2029 12231
12507
2030 12230
12507
2031 12229
12507
2032 12228
12507
2033 12227
12507
2034 12226
12506
2035 12225
12505
2036 12224
12505
2037 12223
12505
2038 12222
12505
2039 12221
12505
2040 12220
12505
2041 12219
12505
2042 12218
12505
2043 12217
12504
2044 12216
12504
2045 12215
12503
2046 12214
12503
2047 12213
12502
2048 12212
12502
2049 12211
12501
2050 12210
12499
2051 12209
12498
2052 122

...

10855
11984 2276
10855
11985 2275
10855
11986 2274
10855
11987 2273
10855
11988 2272
10855
11989 2271
10855
11990 2270
10855
11991 2269
10855
11992 2268
10855
11993 2267
10855
11994 2266
10855
11995 2265
10854
11996 2264
10854
11997 2263
10854
11998 2262
10854
11999 2261
10854
12000 2260
10854
12001 2259
10854
12002 2258
10854
12003 2257
10854
12004 2256
10854
12005 2255
10854
12006 2254
10854
12007 2253
10854
12008 2252
10854
12009 2251
10854
12010 2250
10854
12011 2249
10854
12012 2248
10854
12013 2247
10854
12014 2246
10854
12015 2245
10854
12016 2244
10854
12017 2243
10854
12018 2242
10854
12019 2241
10854
12020 2240
10854
12021 2239
10854
12022 2238
10854
12023 2237
10854
12024 2236
10854
12025 2235
10854
12026 2234
10854
12027 2233
10854
12028 2232
10854
12029 2231
10854
12030 2230
10854
12031 2229
10854
12032 2228
10854
12033 2227
10854
12034 2226
10854
12035 2225
10854
12036 2224
10854
12037 2223
10854
12038 2222
10854
12039 2221
10854
12040 2220
10854
12041 2219
10854
12042 22

10846
12469 1791
10846
12470 1790
10846
12471 1789
10846
12472 1788
10846
12473 1787
10846
12474 1786
10846
12475 1785
10846
12476 1784
10846
12477 1783
10846
12478 1782
10846
12479 1781
10846
12480 1780
10846
12481 1779
10846
12482 1778
10846
12483 1777
10846
12484 1776
10846
12485 1775
10846
12486 1774
10846
12487 1773
10846
12488 1772
10846
12489 1771
10846
12490 1770
10846
12491 1769
10846
12492 1768
10846
12493 1767
10846
12494 1766
10846
12495 1765
10846
12496 1764
10846
12497 1763
10846
12498 1762
10846
12499 1761
10846
12500 1760
10846
12501 1759
10846
12502 1758
10846
12503 1757
10846
12504 1756
10846
12505 1755
10846
12506 1754
10846
12507 1753
10846
12508 1752
10846
12509 1751
10845
12510 1750
10845
12511 1749
10845
12512 1748
10845
12513 1747
10845
12514 1746
10845
12515 1745
10845
12516 1744
10844
12517 1743
10844
12518 1742
10844
12519 1741
10844
12520 1740
10844
12521 1739
10844
12522 1738
10844
12523 1737
10844
12524 1736
10844
12525 1735
10844
12526 1734
10844
12527 17

10835
12952 1308
10835
12953 1307
10835
12954 1306
10835
12955 1305
10835
12956 1304
10835
12957 1303
10835
12958 1302
10835
12959 1301
10835
12960 1300
10835
12961 1299
10835
12962 1298
10835
12963 1297
10835
12964 1296
10835
12965 1295
10835
12966 1294
10835
12967 1293
10835
12968 1292
10835
12969 1291
10835
12970 1290
10835
12971 1289
10835
12972 1288
10835
12973 1287
10835
12974 1286
10835
12975 1285
10835
12976 1284
10835
12977 1283
10835
12978 1282
10835
12979 1281
10835
12980 1280
10835
12981 1279
10835
12982 1278
10835
12983 1277
10835
12984 1276
10835
12985 1275
10835
12986 1274
10835
12987 1273
10835
12988 1272
10835
12989 1271
10835
12990 1270
10835
12991 1269
10835
12992 1268
10835
12993 1267
10835
12994 1266
10835
12995 1265
10835
12996 1264
10835
12997 1263
10835
12998 1262
10835
12999 1261
10835
13000 1260
10835
13001 1259
10835
13002 1258
10835
13003 1257
10835
13004 1256
10835
13005 1255
10835
13006 1254
10834
13007 1253
10834
13008 1252
10834
13009 1251
10834
13010 12

10830
13445 815
10830
13446 814
10830
13447 813
10830
13448 812
10830
13449 811
10830
13450 810
10830
13451 809
10830
13452 808
10830
13453 807
10830
13454 806
10830
13455 805
10830
13456 804
10830
13457 803
10830
13458 802
10830
13459 801
10830
13460 800
10830
13461 799
10830
13462 798
10830
13463 797
10830
13464 796
10830
13465 795
10830
13466 794
10829
13467 793
10829
13468 792
10829
13469 791
10829
13470 790
10829
13471 789
10829
13472 788
10829
13473 787
10829
13474 786
10829
13475 785
10829
13476 784
10829
13477 783
10829
13478 782
10829
13479 781
10829
13480 780
10829
13481 779
10829
13482 778
10829
13483 777
10829
13484 776
10829
13485 775
10829
13486 774
10829
13487 773
10829
13488 772
10829
13489 771
10829
13490 770
10829
13491 769
10829
13492 768
10829
13493 767
10829
13494 766
10829
13495 765
10829
13496 764
10829
13497 763
10829
13498 762
10829
13499 761
10829
13500 760
10828
13501 759
10828
13502 758
10828
13503 757
10828
13504 756
10828
13505 755
10828
13506 754
10828
13

10826
13974 286
10826
13975 285
10826
13976 284
10826
13977 283
10826
13978 282
10826
13979 281
10826
13980 280
10826
13981 279
10826
13982 278
10826
13983 277
10826
13984 276
10826
13985 275
10826
13986 274
10826
13987 273
10826
13988 272
10826
13989 271
10826
13990 270
10826
13991 269
10826
13992 268
10826
13993 267
10826
13994 266
10826
13995 265
10826
13996 264
10826
13997 263
10826
13998 262
10826
13999 261
10826
14000 260
10826
14001 259
10826
14002 258
10826
14003 257
10826
14004 256
10826
14005 255
10826
14006 254
10826
14007 253
10826
14008 252
10826
14009 251
10826
14010 250
10826
14011 249
10826
14012 248
10826
14013 247
10826
14014 246
10826
14015 245
10826
14016 244
10826
14017 243
10826
14018 242
10826
14019 241
10826
14020 240
10826
14021 239
10826
14022 238
10826
14023 237
10826
14024 236
10826
14025 235
10826
14026 234
10826
14027 233
10826
14028 232
10826
14029 231
10826
14030 230
10826
14031 229
10826
14032 228
10826
14033 227
10826
14034 226
10826
14035 225
10826
14

In [36]:
print(len(filteredBedroomImages))
filteredBedroomFeatures = []
for item in filteredBedroomImages:
    for item2 in bedroomPairs:
        if (item in item2[1]):
            filteredBedroomFeatures.append(item2[0])
print(len(filteredBedroomFeatures))

10826
10826


In [35]:
with open('filteredBedroomImages.pkl', 'wb') as f:
    pickle.dump(filteredBedroomImages, f)

In [50]:
#Takes bedroom dataset and filters/keeps the images listed in filteredbedroomimages
bedroomDF = pd.read_csv('/Users/AZG6ELS/Developer/Bedroom/bedroom_images.csv')
filteredBedroomImageIDs = [get_imageID(item) for item in filteredBedroomImages]
print(len(filteredBedroomImageIDs))

10826


In [52]:
bedroomDFFiltered = bedroomDF.loc[bedroomDF['OMS_SKU'].isin(filteredBedroomImageIDs)].drop_duplicates(subset='OMS_SKU')
print(bedroomDFFiltered.shape)

(10826, 11)


In [53]:
bedroomDFFiltered.to_csv('Bedroom/FilteredBedroomSet.csv',index = False)

In [19]:
###filtering out duplicates in a1342998 3.csv which is the bedroom bounding box set for combos
bedroomComboDF = pd.read_csv('roomCombos/a1342998 3.csv')
filteredIDs = [get_imageID(item) for item in filteredImages]
print(len(filteredIDs))
bedroomComboDFFiltered = bedroomComboDF.loc[bedroomComboDF['_unit_id'].isin(filteredIDs)].drop_duplicates(subset='_unit_id')
print(bedroomComboDFFiltered.shape)

2143
(2143, 23)


In [20]:
bedroomComboDFFiltered.to_csv('roomCombos/FilteredBedroomComboSet.csv',index = False)

In [22]:
#####filters out duplicates in the living room set
livingRoomDF = pd.read_csv('roomCombos/living_room_images.csv')
filteredIDs = [get_imageID(item) for item in filteredImages]
print(len(filteredIDs))
livingRoomFilteredDF = livingRoomDF.loc[livingRoomDF['OMS_SKU'].isin(filteredIDs)].drop_duplicates(subset='OMS_SKU')
print(livingRoomFilteredDF.shape)

23737
(23737, 11)


In [23]:
livingRoomFilteredDF.to_csv('roomCombos/FilteredLivingRoomSet2.csv',index=False)

In [18]:
####filters out duplicates in outdoor set as well as stuff already contained in a13... csv
outroomDF = pd.read_csv('OutdoorRooms.csv')
outoldDF = pd.read_csv('a1314054.csv')

filteredIDs = [get_imageID(item) for item in filteredImages]
outroomDF = outroomDF.loc[outroomDF['OMS_SKU'].isin(filteredIDs)].drop_duplicates(subset='OMS_SKU')

In [19]:
outroomDF.shape

(2035, 11)

In [28]:
oldIDs = []
for i,row in enumerate(outoldDF.itertuples()):
    oldIDs.append(row.oms_sku)

In [29]:
oldIDs[0]

100277869

In [32]:
filteredDF = outroomDF.loc[~outroomDF['OMS_SKU'].isin(oldIDs)].drop_duplicates(subset='OMS_SKU')

In [33]:
filteredDF.shape

(1895, 11)

In [34]:
filteredDF.to_csv('FilteredOutdoorRooms.csv',index=False)

In [35]:
###filters out duplicates in bedroom set as well as stuff already contained in original bedroom set
bedroomDF = pd.read_csv('CrowdFlower/FilteredBedroomSet.csv')
bedroomoldDF = pd.read_csv('CrowdFlower/a1342998 3.csv')

oldIDs = []
for i,row in enumerate(bedroomoldDF.itertuples()):
    oldIDs.append(row.oms_sku)
filteredDF = bedroomDF.loc[~bedroomDF['OMS_SKU'].isin(oldIDs)].drop_duplicates(subset='OMS_SKU')
print(filteredDF.shape)

(8692, 11)


In [36]:
filteredDF.to_csv('FilteredBedroomRooms.csv',index=False)

In [54]:
####bathroom - a1308446.csv
bathroomDF = pd.read_csv('CrowdFlower/allbathrooms.csv')
bathroomoldDF = pd.read_csv('CrowdFlower/a1308446.csv')

filteredIDs = [get_imageID(item) for item in filteredImages]
bathroomDF = bathroomDF.loc[bathroomDF['OMS_SKU'].isin(filteredIDs)].drop_duplicates(subset='OMS_SKU')

In [57]:
bathroomDF.to_csv('FilteredBathroomRooms.csv',index=False)

In [39]:
####kitchen - a1315160.csv
kitchenDF = pd.read_csv('CrowdFlower/KitchenRooms.csv')
kitchenoldDF = pd.read_csv('CrowdFlower/a1315160.csv')

filteredIDs = [get_imageID(item) for item in filteredImages]
kitchenDF = kitchenDF.loc[kitchenDF['OMS_SKU'].isin(filteredIDs)].drop_duplicates(subset='OMS_SKU')

oldIDs = []
for i,row in enumerate(kitchenoldDF.itertuples()):
    oldIDs.append(row.oms_sku)
kitchenDF = kitchenDF.loc[~kitchenDF['OMS_SKU'].isin(oldIDs)].drop_duplicates(subset='OMS_SKU')
print(kitchenDF.shape)

(6324, 12)


In [40]:
kitchenDF.to_csv('CrowdFlower/FilteredKitchenRooms.csv',index=False)

In [20]:
####Dining Room
diningroomDF = pd.read_csv('CrowdFlower/DiningRooms.csv')
filteredIDs = [get_imageID(item) for item in filteredImages]
diningroomDF = diningroomDF.loc[diningroomDF['OMS_SKU'].isin(filteredIDs)].drop_duplicates(subset='OMS_SKU')

In [22]:
diningroomDF.to_csv('CrowdFlower/FilteredDiningRooms.csv',index=False)

In [19]:
###
entrywayDF = pd.read_csv('CrowdFlower/EntrywayRooms.csv')
filteredIDs = [get_imageID(item) for item in filteredImages]
entrywayDF = entrywayDF.loc[entrywayDF['OMS_SKU'].isin(filteredIDs)].drop_duplicates(subset='OMS_SKU')

In [21]:
entrywayDF.to_csv('CrowdFlower/FilteredEntrywayRooms.csv',index=False)

In [20]:
entrywayDF.shape

(1328, 11)

In [22]:
entrywayDF

,OMS_SKU,description,top_image_path,Class_Name,Class_Nbr,Dept_Name,Dept_Nbr,discontinued_sku,Taxonomy,attribute_name,alt_image_path
0,303756401,Brayden Chestnut/Black 3-Shelf Console/TV Stan...,https://idm.homedepot.com/assets/image/8c/8caa...,INTERIOR FURNITURE ...,34,BLINDS AND WALLPAPER ...,59,N,Furniture>Living Room Furniture>Accent Tables>...,Lifestyle Image,https://idm.homedepot.com/assets/image/dc/dc3b...
1,305952999,Melodie Red Console Table ...,https://idm.homedepot.com/assets/image/c0/c0ea...,INTERIOR FURNITURE ...,34,BLINDS AND WALLPAPER ...,59,N,Furniture>Living Room Furniture>Accent Tables>...,Lifestyle Image,https://idm.homedepot.com/assets/image/3f/3f2d...
2,303860799,Antonio 46 in. Wide Bench in Yellow Fabric ...,https://idm.homedepot.com/assets/image/8f/8f5a...,INTERIOR FURNITURE ...,34,BLINDS AND WALLPAPER ...,59,N,Furniture>Entryway Furniture>Entryway Benches ...,Lifestyle Image,https://idm.homedepot.com/assets/image/ee/eeb4...
3,308660478,Belaire Chrome and Weathered Gray Console Tabl...,https://idm.homedepot.com/assets/image/95/956e...,INTERIOR FURNITURE ...,34,BLINDS AND WALLPAPER ...,59,N,Furniture>Entryway Furniture>Entryway Tables,Lifestyle Image,https://idm.homedepot.com/assets/image/e5/e59b...
4,303860381,Antonio Cream Velvet Storage Bench with Scroll...,https://idm.homedepot.com/assets/image/37/37cf...,INTERIOR FURNITURE ...,34,BLINDS AND WALLPAPER ...,59,N,Furniture>Entryway Furniture>Entryway Benches ...,Lifestyle Image,https://idm.homedepot.com/assets/image/18/1854...
5,304618861,Quinn Dark Navy Blue Tufted Entryway Bench ...,https://idm.homedepot.com/assets/image/3a/3a04...,INTERIOR FURNITURE ...,34,BLINDS AND WALLPAPER ...,59,N,Furniture>Entryway Furniture>Entryway Benches ...,Lifestyle Image,https://idm.homedepot.com/assets/image/37/37ae...
6,308328173,Capucine 2-Drawer Gray Console Table ...,https://idm.homedepot.com/assets/image/a4/a4bd...,INTERIOR FURNITURE ...,34,BLINDS AND WALLPAPER ...,59,N,Furniture>Entryway Furniture>Entryway Tables,Lifestyle Image,https://idm.homedepot.com/assets/image/2e/2eef...
7,300861584,American Heritage Gray Hall Table ...,https://idm.homedepot.com/assets/image/89/8942...,INTERIOR FURNITURE ...,34,BLINDS AND WALLPAPER ...,59,N,Furniture>Living Room Furniture>Accent Tables>...,Lifestyle Image,https://idm.homedepot.com/assets/image/5c/5ccb...
8,306000659,Medan Warm Brown Console Table ...,https://idm.homedepot.com/assets/image/e5/e5d4...,INTERIOR FURNITURE ...,34,BLINDS AND WALLPAPER ...,59,N,Furniture>Entryway Furniture>Entryway Tables,Lifestyle Image,https://idm.homedepot.com/assets/image/77/7737...
9,205390352,Hanover Burnt Oak and Powder Coated Silver Con...,https://idm.homedepot.com/assets/image/2c/2c10...,INTERIOR FURNITURE ...,34,BLINDS AND WALLPAPER ...,59,N,Furniture>Living Room Furniture>Accent Tables>...,Lifestyle Image,https://idm.homedepot.com/assets/image/95/9547...
